In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# World of Warcraft Log Toy

This notebook toys with data generated from World of Warcraft (WoW) logs.

4 helper classes were created (in the wow folder):
- **wow.log.Record**
  - *Parses and stores an entry of the log*
  - Each line may have differente structure so more classes deriving from this one may be necessary.
  - Each record also represent an in game event.
- **wow.log.Encounter**
  - *A set of records that represents a fight agains a boss in the game*
  - Each starts with a ENCOUNTER_START event and go until an ENCOUNTER_END event is found.
- **wow.query.Query**
  - *An object to help create a chain of iterators to build queries (using filters and map like functions)*
- **wow.query.Predicate**
  - *A Collection of functions that will be applied to those iterators*

There are also another 2 modules in addition to both modules containing classes above:
- **ui**
  - contain ui elements in this notebook

- **fights**
  - contains scripts to analyze each fight (only nerzhul for now)


In [2]:
# Initialize
import pandas as pd
import wow.ui as ui
import wow.helper as help
import wow.fights

from wow.query import Predicate
from wow.log import Encounter, Log

# (z, f) = ui.import_file()


# Create Data Structures

In [3]:
# help.unzip('WoWCombatLog-012722_214646.zip', 'wow')

In [4]:
# file = r'wow\WoWCombatLog-012722_214646.txt'
# file = r'C:\Program Files (x86)\World of Warcraft\_retail_\Logs\RaiderIOLogsArchive\WoWCombatLog-012722_214646.txt'
file = r'C:\Program Files (x86)\World of Warcraft\_retail_\Logs\WoWCombatLog-020122_215917.txt'
log = Log.parse(file)

Encounters:  7


# Detailed Analysis

In [5]:
# Define Encounter being Analysed
encSelect = ui.pick_encounter(log.encounters)

In [21]:
e = log.encounters[encSelect.value]
e.getReport().report()


<style>
sb { color: steelblue }
o { color: Orange }
g { color: Green }
</style>

## <sb>Mythic "Remnant of Ner'zhul" Wipe (id: 2432)</sb>
- 964752 - 2/1 23:55:40.638: ENCOUNTER_START,2432,"Remnant of Ner'zhul",16,20,2450

- 69764 entries in **0:03:21.943000** / 0:03:22.216000
- 1034515 - 2/1 23:59:02.854: ENCOUNTER_END,2432,"Remnant of Ner'zhul",16,20,0,201943



None

,TimeStamp,ID,Player
0,2022-02-01 23:56:07.554000,Creature-0-3887-2450-10650-177117-000079F2B6,"""Djøw-Azralon"""
2,2022-02-01 23:56:12.344000,Creature-0-3887-2450-10650-177117-000079F2B6,"""Thilokz-Azralon"""
1,2022-02-01 23:56:08.417000,Creature-0-3887-2450-10650-177117-000079F2B7,"""Dii-Azralon"""
4,2022-02-01 23:56:56.558000,Creature-0-3887-2450-10650-177117-000079F2DC,"""Thilokz-Azralon"""
5,2022-02-01 23:57:03.899000,Creature-0-3887-2450-10650-177117-000079F2DC,"""Bablowboy-Azralon"""
6,2022-02-01 23:57:07.358000,Creature-0-3887-2450-10650-177117-000079F2DC,"""Dii-Azralon"""
3,2022-02-01 23:56:52.516000,Creature-0-3887-2450-10650-177117-000079F2DE,"""Djøw-Azralon"""
8,2022-02-01 23:57:39.526000,Creature-0-3887-2450-10650-177117-000079F309,"""Maulfurionx-Azralon"""
7,2022-02-01 23:57:35.681000,Creature-0-3887-2450-10650-177117-000079F30B,"""Djøw-Azralon"""
10,2022-02-01 23:58:17.634000,Creature-0-3887-2450-10650-177117-000079F338,"""Pandamän-Azralon"""


,ID,TimeStamp Spawned,TimeStamp Disposed,Time Elapsed,Player
0,Creature-0-3887-2450-10650-177117-000079F2B6,2022-02-01 23:55:51.555000,2022-02-01 23:56:12.344000,0 days 00:00:20.789000,"""Thilokz-Azralon"""
1,Creature-0-3887-2450-10650-177117-000079F2B7,2022-02-01 23:55:53.074000,2022-02-01 23:56:08.417000,0 days 00:00:15.343000,"""Dii-Azralon"""
2,Creature-0-3887-2450-10650-177117-000079F2DC,2022-02-01 23:56:30.027000,2022-02-01 23:57:07.358000,0 days 00:00:37.331000,"""Dii-Azralon"""
3,Creature-0-3887-2450-10650-177117-000079F2DE,2022-02-01 23:56:31.542000,2022-02-01 23:56:52.516000,0 days 00:00:20.974000,"""Djøw-Azralon"""
4,Creature-0-3887-2450-10650-177117-000079F309,2022-02-01 23:57:14.934000,2022-02-01 23:57:39.526000,0 days 00:00:24.592000,"""Maulfurionx-Azralon"""
5,Creature-0-3887-2450-10650-177117-000079F30B,2022-02-01 23:57:16.509000,2022-02-01 23:57:35.681000,0 days 00:00:19.172000,"""Djøw-Azralon"""
6,Creature-0-3887-2450-10650-177117-000079F338,2022-02-01 23:58:02.041000,2022-02-01 23:58:17.634000,0 days 00:00:15.593000,"""Pandamän-Azralon"""
7,Creature-0-3887-2450-10650-177117-000079F33A,2022-02-01 23:58:03.493000,2022-02-01 23:58:21.539000,0 days 00:00:18.046000,"""Djøw-Azralon"""
8,Creature-0-3887-2450-10650-177117-000079F33B,2022-02-01 23:58:05.067000,2022-02-01 23:58:42.196000,0 days 00:00:37.129000,"""Zairoz-Azralon"""


In [12]:
e.timestamp_begin.strftime('encounters_%Y_%m_%d')

'encounters_2022_02_01'

In [ ]:
e.export()

In [20]:
log.save_encounters()

In [ ]:
# Full Log Report
# NerzhulReport(log).listOrbsDisposed().groupby(
#   lambda x: x[1],
#   lambda x: x[2],
#   lambda x: len(list(x))
# ).dict()
# r.listEncounters()

# Combined Logs
# r = EncounterReport(encounters)
# r.listEncounters()


In [ ]:
# import humanize
# from pympler import asizeof
# humanize.naturalsize(
# )
# asizeof.asizeof(log, limit=6, stats=3, cutoff=0)

In [ ]:
(spDmg, mDmg) = e.getReport().getDamage()

dmg = pd.merge(
  spDmg,
  mDmg.drop(['Name'], axis='columns'),
  how='outer',
  on='Unit ID'
)



In [ ]:
dmg['Player ID'] = dmg['Player ID'].fillna(dmg['Unit ID'])
dmg = dmg.fillna(0)
dmg.groupby(['Player ID', 'Name']).sum()

In [ ]:
dmg.join(pd.DataFrame(dmg['Total (Spell)'] + dmg['Total (Melee)'], columns=['Total']) )


In [ ]:
e.getReport().showMeleeDamage()

In [ ]:
e.q.filter(
    Predicate.all([
        Predicate.any([Predicate.isPlayerAction(),
                       Predicate.isPetAction()]),
        Predicate.isTargetHostile(),
        Predicate.isEventIn([
            'SPELL_DAMAGE',
            'SPELL_PERIODIC_DAMAGE',
            'RANGE_DAMAGE'
        ]),
    ])
).map((
    Predicate.getActorId(),
    Predicate.getActor(),
    lambda x: int(x[29]),
    # Predicate.getData(),
)).list()
